# Chapter 21: Smart Contract Upgradeability

---

Smart contracts are immutable by default—once deployed, their code cannot be changed. While immutability is a cornerstone of trust (users can rely on the code they've audited), it also poses challenges. Bugs can be exploited, and new features cannot be added. **Upgradeability** introduces mechanisms to change contract logic while preserving state and address, enabling developers to fix issues and evolve their protocols. However, upgradeability comes with significant complexity and security considerations. In this chapter, we'll explore why upgradeability is needed, the risks involved, and the standard patterns used to implement it safely.

---

## 21.1 The Immutable Challenge

### 21.1.1 Why Upgradeability Matters

**The dilemma:** Deployed contracts are immutable, but software inevitably needs updates—to fix bugs, add features, or respond to changing requirements.

**Reasons for upgrades:**

1. **Critical bug fixes**: A vulnerability discovered after deployment must be patched before it's exploited.
2. **Logic improvements**: Optimize gas usage, improve user experience.
3. **New features**: Add functionality demanded by the community.
4. **Regulatory changes**: Adapt to new legal requirements.
5. **Protocol parameter adjustments**: Change interest rates, fees, etc. (though these can often be managed via setters without full upgrade).

Without upgradeability, the only option is to deploy a new contract and migrate all users and state—a painful and risky process.

### 21.1.2 Risks of Mutable Contracts

Upgradeability introduces trust assumptions and technical risks:

| Risk | Description |
|------|-------------|
| **Centralization** | The upgrade mechanism often gives a single entity (owner) the power to change the rules, undermining decentralization. |
| **Governance attacks** | If upgrade keys are compromised, an attacker can replace the contract with a malicious one. |
| **Storage corruption** | Improper upgrades can corrupt the contract's state (e.g., storage layout mismatches). |
| **Logic inconsistency** | New logic may not align with existing state, leading to unexpected behavior. |
| **User confusion** | Users may not realize the contract's behavior can change, breaking trust. |

Therefore, upgradeability must be implemented with extreme care, following proven patterns and best practices.

---

## 21.2 Proxy Patterns

The core idea behind most upgradeable contracts is the **proxy pattern**: separate logic from state. The user interacts with a proxy contract that stores the data and delegates calls to an implementation contract containing the logic. The proxy's address remains constant; only the implementation address can be updated.

### 21.2.1 Proxy Contract Concept

```
Proxy Pattern Overview:

┌──────────────────┐
│      User        │
└────────┬─────────┘
         │
         ▼
┌──────────────────┐      delegatecall      ┌──────────────────┐
│     Proxy        │────────────────────────▶│  Implementation │
│  - storage       │                         │  - logic        │
│  - admin         │◀────────────────────────│  (v1, v2, ...)  │
│  - implementation│        returns          └──────────────────┘
└──────────────────┘
```

**How it works:**
1. The proxy stores the address of the implementation contract.
2. All calls to the proxy are forwarded to the implementation using `delegatecall`.
3. `delegatecall` executes the implementation's code in the context of the proxy's storage, so state changes affect the proxy's storage.
4. When an upgrade is needed, the admin changes the implementation address in the proxy.

**Key advantage:** The proxy's address is permanent; users always interact with the same address.

### 21.2.2 Implementation vs. Proxy

Understanding the separation is crucial:

- **Proxy**: Holds all state variables. Its storage layout must remain compatible across upgrades.
- **Implementation**: Contains the logic (functions). Does not hold state; its storage layout is used only as a template via `delegatecall`.

**Important:** The implementation contract should not initialize its own state (except via an initializer function) because its storage is never used.

### 21.2.3 Storage Layout Considerations

When using `delegatecall`, the implementation's storage layout is overlaid onto the proxy's storage. This means:

- Variables are accessed at the same storage slots based on their declaration order.
- Adding new variables to the implementation is safe **only if they are appended after all existing variables** (i.e., no reordering or insertion in the middle).
- Removing or reordering variables will corrupt storage.

**Example of safe upgrade:**
```solidity
// v1 implementation
contract V1 {
    uint256 public x;
}

// v2 implementation (safe)
contract V2 {
    uint256 public x;
    uint256 public y;  // appended at the end
}
```

**Unsafe upgrade:**
```solidity
// v1
contract V1 {
    uint256 public x;
    uint256 public y;
}

// v2 (unsafe - reordered)
contract V2 {
    uint256 public y;  // now points to x's slot!
    uint256 public x;
}
```

To prevent storage collisions, upgradeable contracts often use **storage gaps**—placeholder arrays that can be removed in future versions to make room for new variables.

```solidity
contract Base {
    uint256 public baseValue;
    uint256[50] private __gap;  // reserve 50 slots for future base contract variables
}
```

---

## 21.3 Proxy Patterns Deep Dive

Several proxy patterns have emerged, each with trade-offs in complexity and gas costs.

### 21.3.1 Transparent Proxy Pattern

The **Transparent Proxy** pattern uses a single proxy contract that delegates all calls to an implementation, except for administrative functions (like `upgradeTo`). It prevents function selector clashes by checking the caller's role.

**How it works:**
- If the caller is the admin, the proxy executes administrative functions itself (not delegated).
- If the caller is anyone else, all calls are delegated.
- This avoids ambiguity when the implementation has a function with the same selector as an admin function (e.g., `upgradeTo`).

**Simplified transparent proxy (for illustration):**
```solidity
contract TransparentProxy {
    address public implementation;
    address public admin;

    constructor(address _implementation) {
        admin = msg.sender;
        implementation = _implementation;
    }

    modifier ifAdmin() {
        require(msg.sender == admin, "Not admin");
        _;
    }

    function upgradeTo(address newImplementation) external ifAdmin {
        implementation = newImplementation;
    }

    fallback() external payable {
        // If admin, handle locally (but here all admin functions are explicit, so we just delegate)
        require(msg.sender != admin, "Admin cannot delegate");
        _delegate(implementation);
    }

    receive() external payable {
        _delegate(implementation);
    }

    function _delegate(address impl) internal {
        assembly {
            // Copy msg.data.
            calldatacopy(0, 0, calldatasize())
            // Delegatecall.
            let result := delegatecall(gas(), impl, 0, calldatasize(), 0, 0)
            // Copy returned data.
            returndatacopy(0, 0, returndatasize())
            switch result
            case 0 { revert(0, returndatasize()) }
            default { return(0, returndatasize()) }
        }
    }
}
```

**Pros:**
- Simple to understand.
- Avoids selector clashes by reserving admin functions.

**Cons:**
- Higher gas cost because of the admin check on every call.
- Admin address is stored, adding to deployment cost.

### 21.3.2 UUPS (Universal Upgradeable Proxy Standard)

[EIP-1822](https://eips.ethereum.org/EIPS/eip-1822) defines the UUPS pattern, where upgrade logic lives in the implementation, not the proxy. The proxy is minimal and only delegates.

**Proxy (UUPS):**
```solidity
contract UUPSProxy {
    address public implementation;

    constructor(address _implementation) {
        implementation = _implementation;
    }

    fallback() external payable {
        _delegate(implementation);
    }

    receive() external payable {
        _delegate(implementation);
    }

    function _delegate(address impl) internal {
        assembly { /* same as above */ }
    }
}
```

**Implementation with upgrade logic:**
```solidity
import "@openzeppelin/contracts/proxy/utils/UUPSUpgradeable.sol";

contract MyContract is UUPSUpgradeable {
    uint256 public value;

    function initialize(uint256 _value) public initializer {
        __UUPSUpgradeable_init();
        value = _value;
    }

    function _authorizeUpgrade(address newImplementation) internal override onlyOwner {
        // Access control for upgrades
    }
}
```

**Pros:**
- Cheaper per transaction (no admin check in proxy).
- Implementation can control upgrade conditions (e.g., timelock).

**Cons:**
- Risk of forgetting to include upgrade logic, making the contract permanently unupgradeable.
- Implementation must inherit from UUPSUpgradeable and call the initializer.

### 21.3.3 Beacon Proxy Pattern

The **Beacon** pattern introduces an intermediary (the beacon) that stores the implementation address for multiple proxies. This allows upgrading many proxies at once by updating the beacon.

```
Beacon Pattern:

                    ┌─────────────────┐
                    │     Beacon      │
                    │ - implementation│
                    └────────┬────────┘
                             │
            ┌────────────────┼────────────────┐
            │                │                │
            ▼                ▼                ▼
    ┌──────────────┐  ┌──────────────┐  ┌──────────────┐
    │   Proxy 1    │  │   Proxy 2    │  │   Proxy 3    │
    │ (delegates to│  │ (delegates to│  │ (delegates to│
    │  beacon's    │  │  beacon's    │  │  beacon's    │
    │   impl)      │  │   impl)      │  │   impl)      │
    └──────────────┘  └──────────────┘  └──────────────┘
```

**Beacon contract:**
```solidity
contract UpgradeableBeacon is Ownable {
    address private _implementation;

    event Upgraded(address indexed implementation);

    constructor(address implementation_) {
        _setImplementation(implementation_);
    }

    function implementation() public view returns (address) {
        return _implementation;
    }

    function upgradeTo(address newImplementation) public onlyOwner {
        _setImplementation(newImplementation);
    }

    function _setImplementation(address newImplementation) private {
        require(newImplementation != address(0), "Beacon: invalid implementation");
        _implementation = newImplementation;
        emit Upgraded(newImplementation);
    }
}
```

**Beacon proxy:**
```solidity
contract BeaconProxy {
    address private immutable _beacon;

    constructor(address beacon_, bytes memory data_) {
        _beacon = beacon_;
        // Optionally call initializer
        (bool success, ) = _getImplementation().delegatecall(data_);
        require(success, "BeaconProxy: initialization failed");
    }

    function _getImplementation() private view returns (address) {
        return IBeacon(_beacon).implementation();
    }

    fallback() external payable {
        _delegate(_getImplementation());
    }

    receive() external payable {
        _delegate(_getImplementation());
    }
}
```

**Pros:**
- Upgrade many proxies at once (ideal for factory patterns).
- Proxies are cheap to deploy (no need to store implementation address each time).

**Cons:**
- One more hop (beacon) adds a bit of gas overhead.
- Beacon becomes a central point of failure—if compromised, all proxies are at risk.

### 21.3.4 Diamond Pattern (EIP-2535)

The **Diamond** pattern takes upgradeability further by splitting a contract's functionality across multiple implementation contracts (facets) and using a lookup table for function selectors. This allows a single contract to exceed the 24KB contract size limit and upgrade individual functions independently.

**Core concepts:**
- **Diamond**: The proxy that stores state and a mapping of function selectors to facet addresses.
- **Facets**: Implementation contracts, each containing a subset of functions.
- **Diamond Cut**: The operation to add, replace, or remove functions.

**Simplified diamond storage:**
```solidity
contract Diamond {
    struct FacetAddressAndSelectorPosition {
        address facetAddress;
        uint16 selectorPosition;
    }

    mapping(bytes4 => FacetAddressAndSelectorPosition) internal _selectorToFacetAndPosition;
    mapping(address => uint16) internal _facetToSelectorsLength;
    // ... state for ownership, etc.

    fallback() external payable {
        address facet = _selectorToFacetAndPosition[msg.sig].facetAddress;
        require(facet != address(0), "Diamond: function does not exist");
        assembly {
            calldatacopy(0, 0, calldatasize())
            let result := delegatecall(gas(), facet, 0, calldatasize(), 0, 0)
            returndatacopy(0, 0, returndatasize())
            switch result
            case 0 { revert(0, returndatasize()) }
            default { return(0, returndatasize()) }
        }
    }
}
```

**Pros:**
- Exceeds contract size limit by splitting logic.
- Granular upgrades (add/remove functions without redeploying entire logic).
- Can share internal functions and storage across facets.

**Cons:**
- Complex to implement correctly (storage layout must be shared).
- Gas overhead for the function dispatch.
- Tooling is less mature.

---

## 21.4 Implementing Upgradeable Contracts

The most practical way to implement upgradeable contracts is using **OpenZeppelin Upgrades** (a set of plugins for Hardhat and Truffle) that handle the complexities of proxies, storage gaps, and initializers.

### 21.4.1 Using OpenZeppelin Upgrades

**Installation:**
```bash
npm install --save-dev @openzeppelin/hardhat-upgrades @openzeppelin/contracts-upgradeable
npm install --save-dev @nomicfoundation/hardhat-ethers ethers
```

**Hardhat config:**
```javascript
require('@openzeppelin/hardhat-upgrades');

module.exports = {
  solidity: '0.8.19',
};
```

**Writing an upgradeable contract:**
Use the `-upgradeable` contracts from OpenZeppelin, which are designed without constructors (using initializers instead).

```solidity
// contracts/MyContract.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts-upgradeable/proxy/utils/Initializable.sol";
import "@openzeppelin/contracts-upgradeable/access/OwnableUpgradeable.sol";
import "@openzeppelin/contracts-upgradeable/proxy/utils/UUPSUpgradeable.sol";

contract MyContract is Initializable, OwnableUpgradeable, UUPSUpgradeable {
    uint256 public value;

    /// @custom:oz-upgrades-unsafe-allow constructor
    constructor() {
        _disableInitializers(); // prevents initializing implementation contract
    }

    function initialize(uint256 _value) public initializer {
        __Ownable_init();
        __UUPSUpgradeable_init();
        value = _value;
    }

    function setValue(uint256 _value) public onlyOwner {
        value = _value;
    }

    function _authorizeUpgrade(address newImplementation) internal override onlyOwner {}
}
```

**Deployment script (using Hardhat):**
```javascript
// scripts/deploy.js
const { ethers, upgrades } = require("hardhat");

async function main() {
  const MyContract = await ethers.getContractFactory("MyContract");
  const instance = await upgrades.deployProxy(MyContract, [42], { initializer: 'initialize' });
  await instance.waitForDeployment();

  console.log("Proxy deployed to:", await instance.getAddress());
  console.log("Implementation address:", await upgrades.erc1967.getImplementationAddress(await instance.getAddress()));
}

main().catch(console.error);
```

**Upgrade script:**
```javascript
// scripts/upgrade.js
const { ethers, upgrades } = require("hardhat");

async function main() {
  const proxyAddress = "0x..."; // existing proxy address
  const MyContractV2 = await ethers.getContractFactory("MyContractV2");
  const upgraded = await upgrades.upgradeProxy(proxyAddress, MyContractV2);
  console.log("Upgraded");
}
```

The plugin handles:
- Deploying a proxy (Transparent or UUPS, depending on your contract).
- Setting the initial implementation.
- Calling the initializer.
- Storing the admin address (for transparent proxy) or expecting UUPS logic.

### 21.4.2 Storage Gaps

When writing upgradeable contracts, especially with inheritance, you must reserve storage slots to allow future versions to add variables. OpenZeppelin contracts often include a `__gap` array.

```solidity
contract Base {
    uint256 public baseValue;
    uint256[50] private __gap;  // reserve 50 slots for future Base variables
}

contract Child is Base {
    uint256 public childValue;
    uint256[50] private __gap;  // reserve for Child
}
```

When you later add a variable to `Base`, you remove one from the gap, ensuring storage layout remains compatible.

### 21.4.3 Initialization Functions

Because constructors are not used in upgradeable contracts (they would initialize the implementation, not the proxy), we use **initializer functions**. The `initializer` modifier ensures the function can only be called once (like a constructor).

- Call base contract initializers explicitly: `__Ownable_init()`, `__UUPSUpgradeable_init()`.
- Use `initializer` on your own initialize function.
- Never use the `constructor` for logic that should apply to the proxy.

### 21.4.4 Complete Implementation Guide

Let's walk through building a simple upgradeable token contract.

**Step 1: Write the contract (v1)**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts-upgradeable/token/ERC20/ERC20Upgradeable.sol";
import "@openzeppelin/contracts-upgradeable/access/OwnableUpgradeable.sol";
import "@openzeppelin/contracts-upgradeable/proxy/utils/Initializable.sol";
import "@openzeppelin/contracts-upgradeable/proxy/utils/UUPSUpgradeable.sol";

contract MyTokenV1 is Initializable, ERC20Upgradeable, OwnableUpgradeable, UUPSUpgradeable {
    /// @custom:oz-upgrades-unsafe-allow constructor
    constructor() {
        _disableInitializers();
    }

    function initialize(string memory name, string memory symbol, uint256 initialSupply) initializer public {
        __ERC20_init(name, symbol);
        __Ownable_init();
        __UUPSUpgradeable_init();
        _mint(msg.sender, initialSupply);
    }

    function mint(address to, uint256 amount) public onlyOwner {
        _mint(to, amount);
    }

    function _authorizeUpgrade(address newImplementation) internal override onlyOwner {}
}
```

**Step 2: Deploy**
```javascript
// deploy.js
const { ethers, upgrades } = require("hardhat");

async function main() {
  const MyToken = await ethers.getContractFactory("MyTokenV1");
  const token = await upgrades.deployProxy(MyToken, ["MyToken", "MTK", ethers.parseEther("1000")], { initializer: 'initialize' });
  await token.waitForDeployment();
  console.log("Token proxy deployed to:", await token.getAddress());
}
main();
```

**Step 3: Write v2 with new feature (e.g., burn)**
```solidity
// MyTokenV2.sol
import "./MyTokenV1.sol";

contract MyTokenV2 is MyTokenV1 {
    function burn(uint256 amount) public {
        _burn(msg.sender, amount);
    }
}
```

**Step 4: Upgrade**
```javascript
// upgrade.js
const { ethers, upgrades } = require("hardhat");

async function main() {
  const proxyAddress = "0x...";
  const MyTokenV2 = await ethers.getContractFactory("MyTokenV2");
  const upgraded = await upgrades.upgradeProxy(proxyAddress, MyTokenV2);
  console.log("Upgraded");
}
```

The plugin ensures storage compatibility (checks that no variables were inserted before existing ones).

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 21 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Upgradeability is a double-edged sword: enables fixes/features │
│  but introduces centralization and technical risks.             │
│                                                                 │
│  Proxy Patterns:                                               │
│    • Proxy + Implementation separation                         │
│    • delegatecall executes logic in proxy's storage            │
│    • Storage layout must remain compatible across upgrades     │
│                                                                 │
│  Major Proxy Types:                                            │
│    • Transparent Proxy: admin checks avoid selector clashes    │
│    • UUPS: upgrade logic in implementation (cheaper)           │
│    • Beacon Proxy: multiple proxies share one implementation   │
│    • Diamond: multiple facets, exceed size limit               │
│                                                                 │
│  Best Practices:                                               │
│    • Use OpenZeppelin Upgrades for safe deployment             │
│    • Always use initializer functions, not constructors        │
│    • Include storage gaps in base contracts                    │
│    • Test upgrades thoroughly                                  │
│    • Consider governance mechanisms for upgrade authorization  │
│                                                                 │
│  Security:                                                     │
│    • Upgrade authority should be decentralized (multisig, DAO) │
│    • Timelocks allow users to react to upgrades                │
│    • Proxy admin keys must be secured (hardware wallet)        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 22 – DAOs (Decentralized Autonomous Organizations). We'll explore how DAOs enable community governance, voting mechanisms, treasury management, and tools like Aragon and Snapshot, with a step-by-step guide to building a simple DAO.